## building dataset

In [90]:
from torch.utils.data import Dataset
from torch import Tensor
class MyDataset(Dataset):
    def __getitem__(self, index):
        return Tensor([index, 0.]), Tensor([3. * index + 4])
    def __len__(self):
        return 400
training_dataset = MyDataset()
print(len(training_dataset))
print(training_dataset[0])
print(training_dataset[1])

100
(tensor([0., 0.]), tensor([4.]))
(tensor([1., 0.]), tensor([7.]))


## building dataloader

In [91]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(training_dataset, batch_size=4)
for x, y in train_dataloader:
    print(x)
    print(y)
    break

tensor([[0., 0.],
        [1., 0.],
        [2., 0.],
        [3., 0.]])
tensor([[ 4.],
        [ 7.],
        [10.],
        [13.]])


## building model

In [92]:
from torch import nn
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = nn.Linear(2, 1)
    def forward(self, x):
        return self.model(x)
model = MyModel()
print(model)

MyModel(
  (model): Linear(in_features=2, out_features=1, bias=True)
)


## building optimizer

In [93]:
from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=1e-1)

## training

In [94]:
loss_fn = nn.MSELoss()
for epoch in range(100):
    for batch, (x, y) in enumerate(train_dataloader):
        loss = loss_fn(model(x), y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 20 == 19:
        print(f'epoch: {epoch}, loss: {loss.item()}')
        for n, p in model.named_parameters():
            print(f'name, parameters: {n, p}')
        print()

epoch: 19, loss: 0.09063121676445007
name, parameters: ('model.weight', Parameter containing:
tensor([[3.0089, 0.6755]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([3.3611], requires_grad=True))

epoch: 39, loss: 0.010128804482519627
name, parameters: ('model.weight', Parameter containing:
tensor([[3.0028, 0.6755]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([3.7703], requires_grad=True))

epoch: 59, loss: 9.519350714981556e-05
name, parameters: ('model.weight', Parameter containing:
tensor([[3.0003, 0.6755]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([3.9647], requires_grad=True))

epoch: 79, loss: 6.176996976137161e-07
name, parameters: ('model.weight', Parameter containing:
tensor([[3.0000, 0.6755]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([3.9989], requires_grad=True))

epoch: 99, loss: 3.725290298461914e-09
name, parame

## testing

In [95]:
print(model(Tensor([100, 0])))
print(model(Tensor([10000000, 0])))

tensor([304.0002], grad_fn=<AddBackward0>)
tensor([30000054.], grad_fn=<AddBackward0>)


可以看到，这里泛化到`10000000`就有了不小的误差

## finetuning

In [96]:
loss_fn = nn.MSELoss()
for epoch in range(100):
    for batch, (x, y) in enumerate(train_dataloader):
        loss = loss_fn(model(x), y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 20 == 19:
        print(f'epoch: {epoch}, loss: {loss.item()}')
        for n, p in model.named_parameters():
            print(f'name, parameters: {n, p}')
        print()

epoch: 19, loss: 3.725290298461914e-09
name, parameters: ('model.weight', Parameter containing:
tensor([[3.0000, 0.6755]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([3.9999], requires_grad=True))

epoch: 39, loss: 0.0
name, parameters: ('model.weight', Parameter containing:
tensor([[3.0000, 0.6755]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([4.0000], requires_grad=True))

epoch: 59, loss: 0.0
name, parameters: ('model.weight', Parameter containing:
tensor([[3.0000, 0.6755]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([4.0000], requires_grad=True))

epoch: 79, loss: 0.0
name, parameters: ('model.weight', Parameter containing:
tensor([[3.0000, 0.6755]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([4.0000], requires_grad=True))

epoch: 99, loss: 0.0
name, parameters: ('model.weight', Parameter containing:
tensor([[3.0000, 0.6755]

## testing \# 2

In [97]:
print(model(Tensor([100, 0])))
print(model(Tensor([10000000, 0])))

tensor([304.], grad_fn=<AddBackward0>)
tensor([30000004.], grad_fn=<AddBackward0>)
